In [177]:
import pandas as pd
import numpy as np 
import re

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

In [134]:
df = pd.read_csv('C:\\Users\\Eduarda\\Documents\\real_state\\data\\sell.csv')

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567135 entries, 0 to 567134
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   created_on                  567135 non-null  object 
 1   operation                   567135 non-null  object 
 2   property_type               567135 non-null  object 
 3   place_name                  567135 non-null  object 
 4   place_with_parent_names     567135 non-null  object 
 5   geonames_id                 97 non-null      float64
 6   lat-lon                     273006 non-null  object 
 7   lat                         273006 non-null  float64
 8   lon                         273006 non-null  float64
 9   price                       530859 non-null  float64
 10  currency                    530841 non-null  object 
 11  price_aprox_local_currency  530859 non-null  float64
 12  price_aprox_usd             530859 non-null  float64
 13  surface_total_

In [136]:
df.columns

Index(['created_on', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'geonames_id', 'lat-lon', 'lat', 'lon',
       'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd',
       'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2',
       'price_per_m2', 'floor', 'rooms', 'expenses', 'properati_url',
       'description', 'title', 'image_thumbnail', 'state'],
      dtype='object')

## Feature Selection

In [137]:
# No information
cols = ['created_on', 'operation', 'property_type',
        'place_with_parent_names', 'geonames_id', 
        'lat-lon', 'lat', 'lon','properati_url',
        'title', 'image_thumbnail']

df = df.drop(columns=cols, axis=1)

In [138]:
# High correlation with price
cols = ['price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','expenses','surface_total_in_m2']

df = df.drop(columns=cols,axis=1)

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567135 entries, 0 to 567134
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   place_name             567135 non-null  object 
 1   price                  530859 non-null  float64
 2   currency               530841 non-null  object 
 3   surface_covered_in_m2  422816 non-null  float64
 4   price_per_m2           390150 non-null  float64
 5   floor                  49845 non-null   float64
 6   rooms                  350437 non-null  float64
 7   description            567135 non-null  object 
 8   state                  567135 non-null  object 
dtypes: float64(5), object(4)
memory usage: 38.9+ MB


## Data Cleaning

### Correcting inconsistencies

In [140]:
df.currency.value_counts()

currency
BRL    530595
USD       167
MXN        79
Name: count, dtype: int64

In [141]:
df = df[df['currency'] == 'BRL']
df = df.drop(columns='currency', axis=1)

In [142]:
df.state.value_counts()

state
São Paulo              370944
Rio Grande do Sul       47114
Rio de Janeiro          38098
Minas Gerais            18205
Santa Catarina          18155
Paraná                   9183
Ceará                    5388
Bahia                    5351
Espírito Santo           4974
Paraíba                  3467
Rio Grande do Norte      2175
Distrito Federal         1882
Pernambuco               1636
Goiás                    1276
Pará                      649
Alagoas                   414
Mato Grosso               376
Sergipe                   351
Maranhão                  330
Amazonas                  162
Piauí                     135
Tocantins                 126
Miami                      81
Rondônia                   58
Mato Grosso do Sul         52
Acre                        7
Amapá                       6
Name: count, dtype: int64

In [143]:
df = df[df['state'] != 'Miami']

In [144]:
number_map = {
    'um': 1, 'uma': 1, 'dois': 2, 'duas': 2, 'três': 3, 'tres': 3,
    'quatro': 4, 'cinco': 5, 'seis': 6, 'sete': 7, 'oito': 8, 'nove': 9,
    'dez': 10
}
room_pattern = r'(\d+|um|uma|dois|duas|três|tres|quatro|cinco|seis|sete|oito|nove|dez)\s*(quarto|quartos|qtos?|dormitórios?|dorms?|suítes?|suite)'

extract_number_room = (df['description']
                        .str.lower()
                        .str.extract(room_pattern)[0]
                        .str.strip()
                        .replace(number_map)
)

df['rooms'] = df['rooms'].mask(df['rooms'] != extract_number_room, extract_number_room)

In [145]:
############### resover o room com numeros esquisitos

In [146]:
extract_number_room.value_counts().tail(20)

0
682270     1
2897       1
2898       1
0482       1
1550000    1
380003     1
305001     1
63         1
057102     1
150002     1
9563       1
20163      1
647410     1
79213      1
01142      1
057201     1
057302     1
057602     1
110962     1
1600003    1
Name: count, dtype: int64

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 530514 entries, 0 to 567134
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   place_name             530514 non-null  object 
 1   price                  530514 non-null  float64
 2   surface_covered_in_m2  397001 non-null  float64
 3   price_per_m2           390073 non-null  float64
 4   floor                  49566 non-null   float64
 5   rooms                  391381 non-null  object 
 6   description            530514 non-null  object 
 7   state                  530514 non-null  object 
dtypes: float64(4), object(4)
memory usage: 36.4+ MB


In [148]:
df['floor'] = df['floor'].round().astype('Int64')
df['rooms'] = pd.to_numeric(df['rooms'], errors='coerce')

### Handling missing values

In [149]:
df.isna().sum()

place_name                    0
price                         0
surface_covered_in_m2    133513
price_per_m2             140441
floor                    480948
rooms                    139133
description                   0
state                         0
dtype: int64

In [150]:
df['floor'] = df['floor'].fillna(0)

In [151]:
df.groupby('state')['rooms'].median()

state
Acre                   2.0
Alagoas                3.0
Amapá                  2.0
Amazonas               3.0
Bahia                  2.0
Ceará                  3.0
Distrito Federal       2.0
Espírito Santo         3.0
Goiás                  3.0
Maranhão               2.0
Mato Grosso            3.0
Mato Grosso do Sul     2.0
Minas Gerais           3.0
Paraná                 3.0
Paraíba                2.0
Pará                   3.0
Pernambuco             3.0
Piauí                  3.0
Rio Grande do Norte    2.0
Rio Grande do Sul      2.0
Rio de Janeiro         2.0
Rondônia               2.0
Santa Catarina         2.0
Sergipe                2.0
São Paulo              2.0
Tocantins              2.0
Name: rooms, dtype: float64

In [152]:
rooms_median_state = df.groupby('state')['rooms'].transform('median')

df['rooms'] = df['rooms'].fillna(rooms_median_state)

In [153]:
df.isna().sum()

place_name                    0
price                         0
surface_covered_in_m2    133513
price_per_m2             140441
floor                         0
rooms                         0
description                   0
state                         0
dtype: int64

In [154]:
price_m2_median_place = df.groupby('place_name')['price_per_m2'].transform('median')

df['price_per_m2'] = df['price_per_m2'].fillna(price_m2_median_place)

In [155]:
surface_m2_median_place = df.groupby('place_name')['surface_covered_in_m2'].transform('median')

df['surface_covered_in_m2'] = df['surface_covered_in_m2'].fillna(surface_m2_median_place)

In [156]:
df.isna().sum()

place_name                  0
price                       0
surface_covered_in_m2    1467
price_per_m2             1506
floor                       0
rooms                       0
description                 0
state                       0
dtype: int64

In [157]:
df = df.dropna()

In [158]:
df.isna().sum()

place_name               0
price                    0
surface_covered_in_m2    0
price_per_m2             0
floor                    0
rooms                    0
description              0
state                    0
dtype: int64

### Removing duplicates

In [159]:
df.duplicated().sum()

np.int64(21381)

In [160]:
df = df.drop_duplicates()

### Removing outliers

In [161]:
df.columns

Index(['place_name', 'price', 'surface_covered_in_m2', 'price_per_m2', 'floor',
       'rooms', 'description', 'state'],
      dtype='object')

In [162]:
numeric_cols = ['price', 'surface_covered_in_m2', 'price_per_m2','floor', 'rooms']

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378411 entries, 0 to 567134
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   place_name             378411 non-null  object 
 1   price                  378411 non-null  float64
 2   surface_covered_in_m2  378411 non-null  float64
 3   price_per_m2           378411 non-null  float64
 4   floor                  378411 non-null  Int64  
 5   rooms                  378411 non-null  float64
 6   description            378411 non-null  object 
 7   state                  378411 non-null  object 
dtypes: Int64(1), float64(4), object(3)
memory usage: 26.3+ MB


## Data Transformation

### Encoding categorical variables

In [128]:
one_hot = OneHotEncoder(sparse_output=False, drop=None)

state_encoded = one_hot.fit_transform(df[['state']])

column_names = one_hot.get_feature_names_out(['state'])

encoded_df = pd.DataFrame(state_encoded, columns=column_names, index=df.index)

df = pd.concat([df.drop('state', axis=1), encoded_df], axis=1)


In [129]:
df.head()

,place_name,price,surface_covered_in_m2,price_per_m2,floor,rooms,description,state_Acre,state_Alagoas,state_Amapá,...,state_Pernambuco,state_Piauí,state_Rio Grande do Norte,state_Rio Grande do Sul,state_Rio de Janeiro,state_Rondônia,state_Santa Catarina,state_Sergipe,state_São Paulo,state_Tocantins
0,Mondubim,155900.0,55.0,2892.218182,0,2.0,"Otimo Imovel com o melhor valor da regiao, con...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,Ponta da Praia,230000.0,112.0,5454.545455,0,1.0,"Otimo Imovel com o melhor valor da regiao, con...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
27,São Vicente,195000.0,65.0,4042.553191,0,2.0,"Otimo Imovel com o melhor valor da regiao, con...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
28,São Vicente,400000.0,65.0,4042.553191,0,2.0,"Otimo Imovel com o melhor valor da regiao, con...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
29,São Vicente,240000.0,65.0,4042.553191,0,2.0,"Otimo Imovel com o melhor valor da regiao, con...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [170]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1),
                                                    df['price'],
                                                    test_size=0.2,
                                                    random_state=42)

In [171]:
# target encoding

mean_price_by_place = X_train.join(y_train).groupby('place_name')['price'].mean()
X_train['place_encoded'] = X_train['place_name'].map(mean_price_by_place)

In [172]:
global_mean = y_train.mean()
X_test['place_encoded'] = X_test['place_name'].map(mean_price_by_place).fillna(global_mean)

### Normalization/Standardization

In [178]:
X_train.columns

Index(['place_name', 'surface_covered_in_m2', 'price_per_m2', 'floor', 'rooms',
       'description', 'state', 'place_encoded'],
      dtype='object')

In [ ]:
num_cols = ['surface_covered_in_m2', 'price_per_m2', 'floor', 'rooms', 'place_encoded']

X_train_num = X_train[num_cols]
X_test_num = X_test[num_cols]

In [ ]:
scaler = RobustScaler()

X_train_scaled = scaler.fit_transform(X_train_num)
X_test_scaled = scaler.transform(X_test_num)

### Feature engineering

### Dimensionality reduction